In [13]:
from locale import Error
import tensorflow as tf
import tensorflow as tf
import numpy as np

from turtle import up
import pandas as pd
import scipy
from scipy.stats import iqr


def windowed_dataset(dataset: tf.data.Dataset, window_size=5, shift=1, stride=2):
    if window_size == None or window_size <= 0:
        return dataset

    def sub_to_batch(t1, t2=None):
        t1_batches = t1.batch(window_size, drop_remainder=True)

        if t2 is not None:
            t2_batches = t2.batch(window_size, drop_remainder=True)
            return tf.data.Dataset.zip((t1_batches, t2_batches))
        else:
            return t1_batches

    def pick_batch(t1, t2: tf.Tensor | None):
        #return (t1, t2)
        if t2 is None:
            return t1
        else:

            # 获取张量的维度信息
            ndim = 0 if t2.ndim is None else t2.ndim

            # 确保张量至少有二维
            if ndim >= 2:
                # 获取第二维的大小
                second_dim_size = t2.shape[1] if ndim > 2 else t2.shape[0]
                # 提取每个第二维元素的第一个数
                # 如果张量是二维的，直接使用 tensor[:, 0]
                # 如果张量大于二维，确保正确处理其他维度
                if ndim > 2:
                    first_elements = t2[0, ...]  # 使用...来保持其余维度不变
                else:  # 对于二维张量
                    first_elements = t2[0]

                print(first_elements.shape)
                return t1, first_elements
            else:
                msg = "The tensor does not have at least two dimensions to extract elements from the second dimension."
                print(msg)
                Error(msg)

    return dataset.window(window_size, shift=shift, stride=stride, drop_remainder=True).flat_map(sub_to_batch).map(pick_batch)
    return windows  #.batch(window_size, drop_remainder=drop_remainder)


# def windowed_batch_dataframe(df: pd.DataFrame, window_size=4, batch_size=None):
#     df = df.copy()
#     if batch_size is not None and batch_size > 0:
#         dataset = windowed_dataset(tf.data.Dataset.from_tensor_slices(df.values), window_size).batch(batch_size=batch_size, drop_remainder=True)
#     else:
#         dataset = windowed_dataset(tf.data.Dataset.from_tensor_slices(df.values), window_size)
#     return dataset


def windowed_batch_dataset(ds: tf.data.Dataset, window_size=4, batch_size=None):

    if batch_size is not None and batch_size > 0:
        dataset = windowed_dataset(ds, window_size).batch(batch_size=batch_size, drop_remainder=True)
    else:
        dataset = windowed_dataset(ds, window_size)
    return dataset


def convert_to_numpy(ds: tf.data.Dataset):
    # 将 dataset 转换为 NumPy 数组
    numpy_array = None
    for data in ds.as_numpy_iterator():
        # 确保数据是 NumPy 数组
        data_np = np.array(data)

        # 如果 numpy_array 还没有初始化，初始化它
        if numpy_array is None:
            numpy_array = data_np
        else:
            # # 确保数据的形状与 numpy_array 相同
            # if data_np.shape[1:] != numpy_array.shape[1:]:
            #     raise ValueError("All data items must have the same shape except for the first dimension.")

            # 垂直堆叠数据
            numpy_array = np.vstack((numpy_array, data_np))

    return numpy_array


def get_shape(*args):
    shapes = []
    for arg in args:
        shapes.append(arg.shape if arg is not None else None)
    return shapes

In [14]:
rows = 20
class_num=3
feature_cols = ['a', 'b']
label_cols = ['label']
columns = feature_cols + label_cols  #['a', 'b', 'c']
cols = len(columns)
data_x = np.array([[i + 10 for c in np.arange(1, cols)] + [i % class_num] for i in np.arange(1, rows + 1)])

df = pd.DataFrame(data_x, columns=columns)

print(df)


     a   b  label
0   11  11      1
1   12  12      2
2   13  13      0
3   14  14      1
4   15  15      2
5   16  16      0
6   17  17      1
7   18  18      2
8   19  19      0
9   20  20      1
10  21  21      2
11  22  22      0
12  23  23      1
13  24  24      2
14  25  25      0
15  26  26      1
16  27  27      2
17  28  28      0
18  29  29      1
19  30  30      2


In [15]:
win_size = 4
batch_size = 3


def splice(a, b):
    print('---------')
    print(a)
    return (a, b)


def one_hot_encoding(label):
    # return label
    return tf.squeeze(tf.one_hot(label, depth=class_num))


ds_x = tf.data.Dataset.from_tensor_slices((df[feature_cols]))
ds_y = tf.data.Dataset.from_tensor_slices((df[label_cols])).map(one_hot_encoding)
# ds = tf.data.Dataset.from_tensor_slices((df[feature_cols], df[label_cols]))
ds = tf.data.Dataset.zip((ds_x, ds_y))
win_ds = windowed_batch_dataset(ds, win_size, batch_size)  #.map(splice)
for x, y in win_ds:
    _ = 1
for x, y in win_ds:
    print('************')
    print('')
    print('x--')
    # print(x)
    print(x.shape)
    print('y ==')
    print(tf.argmax(y))
    print('')
    print('')
# print('x1 ===')
# print(y)
# print('===')
# print(y)

# a, b = convert_to_numpy(x), convert_to_numpy(y)
# print(get_shape(a))
# print(a)

(3,)
************

x--
(3, 4, 2)
y ==
tf.Tensor([2 0 1], shape=(3,), dtype=int64)


************

x--
(3, 4, 2)
y ==
tf.Tensor([2 0 1], shape=(3,), dtype=int64)


************

x--
(3, 4, 2)
y ==
tf.Tensor([2 0 1], shape=(3,), dtype=int64)


************

x--
(3, 4, 2)
y ==
tf.Tensor([2 0 1], shape=(3,), dtype=int64)


